# Виконання

## Основне завдання

In [4]:
import numpy as np
import pandas as pd
import nltk
import re
df = pd.read_csv('bbc-news-data.csv', sep='\t')
df

FileNotFoundError: [Errno 2] No such file or directory: 'bbc-news-data.csv'

*Зчитування файлу*

### Видалимо колонки 'filename', 'title', 'category'.

In [ ]:
df.drop(['filename', 'title', 'category'], axis=1, inplace=True)
df

,content
0,Quarterly profits at US media giant TimeWarne...
1,The dollar has hit its highest level against ...
2,The owners of embattled Russian oil giant Yuk...
3,British Airways has blamed high fuel prices f...
4,Shares in UK drinks and food firm Allied Dome...
...,...
2220,BT is introducing two initiatives to help bea...
2221,Computer users across the world continue to i...
2222,A new European directive could put software w...
2223,The man making sure US computer networks are ...


*Видалення колонок*

### Видалимо порожні документи, якщо вони є.

In [ ]:
df = df[~(df.content.str.strip() == '')]
df

,content
0,Quarterly profits at US media giant TimeWarne...
1,The dollar has hit its highest level against ...
2,The owners of embattled Russian oil giant Yuk...
3,British Airways has blamed high fuel prices f...
4,Shares in UK drinks and food firm Allied Dome...
...,...
2220,BT is introducing two initiatives to help bea...
2221,Computer users across the world continue to i...
2222,A new European directive could put software w...
2223,The man making sure US computer networks are ...


*Видалення порожніх документів*

### Визначимо стоп-слова англійської мови.

In [ ]:
wpt = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words('english')

*Стоп-слова*

### Визначимо функцію, що виконує попередню обробку документу. Застосуємо декоратор np.vectorize для того, щоб функція могла працювати з корпусами.

In [ ]:
@np.vectorize
def preproc_doc(doc):
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I | re.A)
    doc = doc.lower()
    doc = doc.strip()
    tokens = wpt.tokenize(doc)
    filtered_tokens = [token for token in tokens if token not in stop_words]
    doc = ' '.join(filtered_tokens)
    return doc

p_corpus = preproc_doc(df.content)
p_corpus

array(['quarterly profits us media giant timewarner jumped bn three months december yearearlier firm one biggest investors google benefited sales highspeed internet connections higher advert sales timewarner said fourth quarter sales rose bn bn profits buoyed oneoff gains offset profit dip warner bros less users aol time warner said friday owns searchengine google internet business aol mixed fortunes lost subscribers fourth quarter profits lower preceding three quarters however company said aols underlying profit exceptional items rose back stronger internet advertising revenues hopes increase subscribers offering online service free timewarner internet customers try sign aols existing customers highspeed broadband timewarner also restate results following probe us securities exchange commission sec close concluding time warners fourth quarter profits slightly better analysts expectations film division saw profits slump helped boxoffice flops alexander catwoman sharp contrast yearearli

*Обробка документів*

### Розіб'ємо кожний документ на окремі слова, об'єднаємо усі слова в одну сукупність.

In [ ]:
df_words = []
for doc in p_corpus:
    df_words.extend(doc.split(' '))
df_words = pd.DataFrame(set(df_words))
df_words.columns = ['words']
df_words.head(10)

,words
0,freeflowing
1,foxed
2,weak
3,recourses
4,christie
5,cigarettes
6,shiner
7,sahundi
8,dawns
9,zodiac


*Розбиття на слова*

### Визначимо частину мови для кожного слова. Використаємо функцію nltk.pos_tag.

In [ ]:
df_words['ps'] = [tag for _, tag in nltk.pos_tag(df_words.words)]
df_words

,words,ps
0,freeflowing,VBG
1,foxed,JJ
2,weak,JJ
3,recourses,NNS
4,christie,VBP
...,...,...
31333,telecast,VBP
31334,orqueras,JJ
31335,eavesdrop,NN
31336,redesigning,VBG


*Визначення частини мови*

### Узагальнимо частини мови до звичайних: noun, adective, verb тощо.

In [ ]:
from nltk.tag import map_tag
df_words['sps'] = [map_tag('en-ptb', 'universal', tag) for tag in df_words.ps]
df_words

,words,ps,sps
0,freeflowing,VBG,VERB
1,foxed,JJ,ADJ
2,weak,JJ,ADJ
3,recourses,NNS,NOUN
4,christie,VBP,VERB
...,...,...,...
31333,telecast,VBP,VERB
31334,orqueras,JJ,ADJ
31335,eavesdrop,NN,NOUN
31336,redesigning,VBG,VERB


*Узагальнення чатин мов*

### Перетворимо узагальнені частини мови на абревіатури для лематизації.

In [ ]:
abbr = {'NOUN': 'n', 'VERB': 'v', 'ADJ': 'a', 'ADV': 'r'}
def to_abbr(el):
    res = abbr.get(el)
    if res is not None:
        return res
    return ''

df_words['abbr'] = [to_abbr(x) for x in df_words.sps]
df_words

,words,ps,sps,abbr
0,freeflowing,VBG,VERB,v
1,foxed,JJ,ADJ,a
2,weak,JJ,ADJ,a
3,recourses,NNS,NOUN,n
4,christie,VBP,VERB,v
...,...,...,...,...
31333,telecast,VBP,VERB,v
31334,orqueras,JJ,ADJ,a
31335,eavesdrop,NN,NOUN,n
31336,redesigning,VBG,VERB,v


*Приведення загальних частин мов до абревіатур*

### Проведемо лематизацію кожного слова за допомогою методу lemmatize об'єкта класу nltk.stem.WordNetLemmatizer.

In [ ]:
from nltk.stem import WordNetLemmatizer
wlem = WordNetLemmatizer()
def my_lem(word, abbr):
    if abbr == '':
        return wlem.lemmatize(word)
    return wlem.lemmatize(word, pos=abbr)
df_words['lemms'] = [my_lem(row[1]['words'], row[1]['abbr']) 
               for row in df_words.loc[:, ['words', 'abbr']].iterrows()]
df_words

,words,ps,sps,abbr,lemms
0,freeflowing,VBG,VERB,v,freeflowing
1,foxed,JJ,ADJ,a,foxed
2,weak,JJ,ADJ,a,weak
3,recourses,NNS,NOUN,n,recourse
4,christie,VBP,VERB,v,christie
...,...,...,...,...,...
31333,telecast,VBP,VERB,v,telecast
31334,orqueras,JJ,ADJ,a,orqueras
31335,eavesdrop,NN,NOUN,n,eavesdrop
31336,redesigning,VBG,VERB,v,redesign


*Лематизація слів*

### Представимо корпус як модуль "Сумка слів". Використаємо для цього клас CountVectorizer зі sklearn.feature_extraction.text.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(min_df=0., max_df=1.)
cv_matrix = cv.fit_transform(p_corpus)
cv_matrix = pd.DataFrame(cv_matrix.toarray(), 
                         columns=cv.get_feature_names_out())
cv_matrix.to_csv('bag_of_words.csv')

*Сумка слів*

### Представимо корпус як модель TD-IDF. ### Перетворимо матрицю з частотою термінів на матрицю tfidf.

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tt = TfidfTransformer(norm='l2', use_idf=True)
tt_matrix = tt.fit_transform(cv_matrix)
tt_matrix = tt_matrix.toarray()
vocab = cv.get_feature_names_out()
tv_matrix = pd.DataFrame(np.round(tt_matrix, 2), columns=vocab)
tv_matrix

,00,000,05,10,100,11,12,125,13,14,...,zooropa,zornotza,zorro,zubair,zuluaga,zurich,zurichs,zutons,zvonareva,zvyagintsev
0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2220,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2221,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2222,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2223,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


*Матриця TF-IDF*

In [ ]:
tv_matrix.